In [1]:
from sklearn.metrics import *
from IPython.core.display import HTML

import sys
import os
import time
import zipfile
import datetime
import glob
import pandas as pd
import numpy as np

# H2O AutoML

In [2]:
import h2o
from h2o.automl import H2OAutoML

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 53 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_Abhinav_k0frpt
H2O cluster total nodes:,1
H2O cluster free memory:,2.490 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [4]:
training_frame = h2o.import_file('historical_data1_Q12005.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
training_frame = training_frame.head(10000)

In [14]:
training_frame.describe()

Rows:10000
Cols:27




,fico,dt_first_pi,flag_fthb,dt_matr,cd_msa,mi_pct,cnt_units,occpy_sts,cltv,dti,orig_upb,ltv,int_rt,channel,ppmt_pnlty,prod_type,st,prop_type,zipcode,id_loan,loan_purpose,orig_loan_term,cnt_borr,seller_name,servicer_name,flag_sc,Year
type,int,int,enum,int,int,int,int,enum,int,int,int,int,real,enum,enum,enum,enum,enum,int,string,enum,int,int,enum,enum,enum,int
mins,487.0,200501.0,,203008.0,0.0,0.0,1.0,,8.0,2.0,20000.0,8.0,4.875,,,,,,600.0,NaN,,305.0,1.0,,,,2005.0
mean,726.4062999999998,200503.08509999973,,203500.90929999988,22347.025399999937,4.879100000000003,1.0330000000000035,,74.17049999999938,49.85770000000005,160097.30000000133,73.0383000000002,5.809322199999983,,,,,,48512.24999999999,NaN,,359.8713999999997,1.6235000000000008,,,,2005.0
maxs,9999.0,200912.0,,203504.0,49740.0,40.0,4.0,,999.0,999.0,648000.0,999.0,8.25,,,,,,99900.0,NaN,,360.0,99.0,,,,2005.0
sigma,214.49341846072187,4.110832991592619,,17.812630420297804,16603.632369579314,10.575285663124443,0.234342679956436,,18.525078882551586,112.84855307137543,80726.87604237828,18.24974351665922,0.2509829241800769,,,,,,26950.33671408201,NaN,,2.058466849035193,2.009365946051273,,,,0.0
zeros,0,0,,0,2601,8138,0,,0,0,0,0,0,,,,,,0,0,,0,0,,,,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,699.0,200505.0,N,203504.0,39300.0,0.0,1.0,P,56.0,42.0,190000.0,56.0,5.625,R,N,FRM,RI,SF,2800.0,F105Q1000001,C,360.0,2.0,Other sellers,USBANKNA,N,2005.0
1,691.0,200504.0,N,203503.0,36420.0,25.0,1.0,P,90.0,36.0,90000.0,90.0,5.75,R,N,FRM,OK,SF,73000.0,F105Q1000002,N,360.0,1.0,Other sellers,Other servicers,N,2005.0
2,713.0,200503.0,N,203502.0,28740.0,0.0,1.0,P,72.0,45.0,357000.0,72.0,6.0,R,N,FRM,NY,SF,12500.0,F105Q1000003,P,360.0,2.0,Other sellers,Other servicers,N,2005.0


In [15]:
testing_frame = h2o.import_file('historical_data1_Q22005.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
testing_frame = testing_frame.head(10000)

In [17]:
X = training_frame.columns
y = 'int_rt'
X.remove(y)

#Unnecessary Columns
X.remove('Year')
X.remove('servicer_name')
X.remove('seller_name')
X.remove('id_loan')
X.remove('zipcode')

In [18]:
aml = H2OAutoML(max_runtime_secs=300)

In [19]:
aml.train(x = X, y = y, training_frame = training_frame, leaderboard_frame= testing_frame)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [20]:
aml.leaderboard.as_data_frame()

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,DeepLearning_grid_1_AutoML_20181128_154512_mod...,0.098466,0.313793,0.098466,0.230591,0.044139
1,GBM_grid_1_AutoML_20181128_154512_model_2,0.105210,0.324362,0.105210,0.249206,0.045785
2,StackedEnsemble_BestOfFamily_AutoML_20181128_1...,0.105809,0.325283,0.105809,0.251722,0.045968
3,GBM_grid_1_AutoML_20181128_154512_model_15,0.107560,0.327964,0.107560,0.251295,0.046274
4,GLM_grid_1_AutoML_20181128_154512_model_1,0.108196,0.328932,0.108196,0.249732,0.046420
5,GBM_grid_1_AutoML_20181128_154512_model_12,0.108485,0.329371,0.108485,0.250015,0.046416
6,GBM_grid_1_AutoML_20181128_154512_model_14,0.108493,0.329383,0.108493,0.255347,0.046558
7,GBM_grid_1_AutoML_20181128_154512_model_17,0.108790,0.329833,0.108790,0.256734,0.046699
8,GBM_grid_1_AutoML_20181128_154512_model_11,0.109393,0.330745,0.109393,0.257228,0.046781
9,GBM_5_AutoML_20181128_154512,0.109617,0.331085,0.109617,0.257421,0.046793


# TPOT

In [2]:
from tpot import TPOTRegressor

In [3]:
tpot = TPOTRegressor(generations=2, population_size=50,
                     offspring_size=None,
                     mutation_rate=0.9,
                     verbosity=3,cv=2,n_jobs=-1)

In [4]:
df_train = pd.read_csv('historical_data1_Q12005.csv',low_memory=False)

In [5]:
df_test = pd.read_csv('historical_data1_Q22005.csv',low_memory=False)

In [6]:
def checkAllReqColumns(df):
    cols_to_keep = ['fico','flag_fthbN','flag_fthbX','flag_fthbY','mi_pct','cnt_units','occpy_stsl','occpy_stsO','occpy_stsS','cltv','dti','orig_upb','ltv','int_rt','channelB','channelC','channelR','channelT','ppmt_pnltyN','ppmt_pnltyX','ppmt_pnltyY','prop_typeCO','prop_typeCP','prop_typeLH','prop_typeMH','prop_typePU','prop_typeSF','prop_typeXX','loan_purposeC','loan_purposeN','loan_purposeP','orig_loan_term','cnt_borr']
    
    for x in cols_to_keep:
        if not x in df.columns:
            df[x] = 0.0
            
    df = df._get_numeric_data()
    df.drop('cd_msa',axis=1,inplace=True)
    df.drop('dt_first_pi',axis=1,inplace=True)
    df.drop('dt_matr',axis=1,inplace=True)
    df.drop('flag_sc',axis=1,inplace=True)
    df.drop('zipcode',axis=1,inplace=True)
    
    return df

In [7]:
def prepare_data_for_model(df):
    dummies1 = pd.get_dummies(df['flag_fthb']).rename(columns=lambda x: 'flag_fthb' + str(x))
    train_df = pd.concat([df, dummies1], axis=1)
    
    dummies2 = pd.get_dummies(df['occpy_sts']).rename(columns=lambda x: 'occpy_sts' + str(x)) 
    train_df = pd.concat([train_df, dummies2], axis=1)
    
    dummies3 = pd.get_dummies(df['channel']).rename(columns=lambda x: 'channel' + str(x)) 
    train_df = pd.concat([train_df, dummies3], axis=1)
    
    dummies4 = pd.get_dummies(df['ppmt_pnlty']).rename(columns=lambda x: 'ppmt_pnlty' + str(x)) 
    train_df = pd.concat([train_df, dummies4], axis=1)
    
    dummies5 = pd.get_dummies(df['prop_type']).rename(columns=lambda x: 'prop_type' + str(x)) 
    train_df = pd.concat([train_df, dummies5], axis=1)
    
    dummies6 = pd.get_dummies(df['loan_purpose']).rename(columns=lambda x: 'loan_purpose' + str(x)) 
    train_df = pd.concat([train_df, dummies6], axis=1)
    
    train_df['flag_sc'] = train_df['flag_sc'].map({'Y':1,'N':0})
    
    train_df = checkAllReqColumns(train_df)
    
    return train_df

In [8]:
processed_train = prepare_data_for_model(df_train)
processed_test = prepare_data_for_model(df_test)

In [9]:
X_train = processed_train.drop('int_rt',axis=1)
y_train = processed_train['int_rt']

X_test = processed_test.drop('int_rt',axis=1)
y_test = processed_test['int_rt']

In [10]:
tpot.fit(X_train, y_train)

28 operators have been imported by TPOT.


Skipped pipeline #8 due to time out. Continuing to the next pipeline.
Skipped pipeline #15 due to time out. Continuing to the next pipeline.
Skipped pipeline #21 due to time out. Continuing to the next pipeline.
Skipped pipeline #23 due to time out. Continuing to the next pipeline.
Skipped pipeline #26 due to time out. Continuing to the next pipeline.
Skipped pipeline #36 due to time out. Continuing to the next pipeline.
Skipped pipeline #46 due to time out. Continuing to the next pipeline.
Skipped pipeline #50 due to time out. Continuing to the next pipeline.
Skipped pipeline #52 due to time out. Continuing to the next pipeline.
Skipped pipeline #57 due to time out. Continuing to the next pipeline.
Skipped pipeline #59 due to time out. Continuing to the next pipeline.
Skipped pipeline #61 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _random_mut

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=2,
       disable_update_check=False, early_stop=None, generations=2,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=-1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=50,
       random_state=None, scoring=None, subsample=1.0, use_dask=False,
       verbosity=3, warm_start=False)

In [11]:
print('The RMSE of TPOT Regressor is {}'.format(tpot.score(X_test,y_test)))

The RMSE of TPOT Regressor is -0.10542476070776394


In [12]:
tpot.export('tpot_freddiemac_pipeline.py')

True